In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

duckduckgo = DuckDuckGoSearchRun( description="Use this to search any questions or queries asked by user that need to use search engine")

for chunk in duckduckgo.stream("what is the current weather in salem, tamilnadu, india"):
    print(chunk)

Be prepared with the most accurate 10-day forecast for Salem , Tamil Nadu , India with highs, lows, chance of precipitation from The Weather Channel and Weather .com. Weather in Salem for 10 days. Week weather forecast in Salem . Salem , Tamil Nadu Weather Today: Forecast & Temperature. Salem weather today shows a temperature of 16°C with Partly Cloudy conditions. The current weather in Salem feels like 16°C, with humidity at 84%, wind blowing at 8.6 km/h, and a UV Index of 0. Visibility stands at 10 km. Today Salem Tamil Nadu India : Mist with a temperature of 20°C and a wind East-North-East speed of 5 Km/h. The humidity will be 94% and there will be 0.0 mm of precipitation.( Be the first to rate). 15 Day Weather Forecast for Salem . What is the current weather in Salem ? Salem is a large city in Salem district, located on the banks of Thirumanimutharu River in the Indian state of Tamil Nadu . It is 186 kilometres southeast of Bangalore and about 340 kilometres southwest of the state 

In [2]:
from langchain.tools import tool

@tool
def send_email(text : str) -> str:
    """Send emails. Use ONLY when user asks to send an email/gmail."""
    return "Email sent"

@tool(return_direct=True)
def get_datetime() -> str:
    """Get current date and time. Use when user asks for current date/time."""
    from datetime import datetime
    return datetime.now().strftime("%d %B %Y, %I:%M %p")

In [3]:

import os
# from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()
os.getenv("GROQ_API_KEY")
from langchain.chat_models import init_chat_model

llm = init_chat_model("groq:llama-3.1-8b-instant")

llm.invoke("hey")

AIMessage(content="How's it going? Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 36, 'total_tokens': 57, 'completion_time': 0.018348168, 'completion_tokens_details': None, 'prompt_time': 0.009742534, 'prompt_tokens_details': None, 'queue_time': 0.046954616, 'total_time': 0.028090702}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdc77-a6cd-79c2-bacd-5bf45acbf021-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 36, 'output_tokens': 21, 'total_tokens': 57})

In [ ]:
# from langchain.agents import create_agent
import requests
load_dotenv()

token = os.getenv("ACCESS_TOKEN_WHATSAPPP")
# print(token)
@tool
def send_whatsapp_template(to : str, message: str) -> str:
    
    """Send a WhatsApp TEXT message. 
Inputs: to (phone number), message (text body)."""
    
    print(to, message)
    try:
        url = "https://graph.facebook.com/v22.0/962571180271822/messages"
        data = {
  "messaging_product": "whatsapp",
  "to": to,
  "type": "template",
  "template": {
    "name": "hello_world",
    "language": { "code": "en_US" }                                                                 
  }                                                                 
}                                                                                                           

        res = requests.post(url=url, headers={
            "Authorization" : f"Bearer {token}",
            "Content-Type": "application/json"
        }, json=data)
        
        print("PAYLOAD:", data) 
        
        if res.status_code == 200:
            return f"Message sent to {to}"
        else:
            return f"Failed: {res.status_code} | {res.text}"
        
        # return f"Message send to {to}"
    
    except Exception as e:
        return f"Error in sending message {e}"
        
    
    
    
    

In [ ]:
@tool
def send_instagram_message(recipientid : str, message : str) -> str:
    """
    Send an Instagram DM (TEXT).
    Inputs:
      - recipientid: Instagram participant ID (ex: 837220055955920)
      - message: text content
    """
    
    recipientid = str(recipientid)
    
    url = "https://graph.facebook.com/v20.0/937924569410235/messages"
    payload = {
    "recipient": {
        "id": recipientid
    },
    "message": {
        "text": message
    }
}
    headers = {
        "Authorization": f"Bearer {os.getenv('PAGE_TOKEN_INSTA')}",
        "Content-Type": "application/json"
    }
    
    res = requests.post(url, headers=headers, json=payload)
    
    if res.status_code == 200:
            return f"Message sent to {recipientid}"
    else:
            return f"Failed: {res.status_code} | {res.text}"
        
    

In [6]:
toolKit = [duckduckgo, send_email, get_datetime, send_whatsapp_template, send_instagram_message]
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=toolKit,
    
)

In [7]:
example_query = "send whatsapp message to the 919043402788 sending the message as 'Hello there from agent'"

for mode, event in agent.stream(
    {"messages": [("user", example_query)]},
    stream_mode=["updates"],
):
    print(mode, event)
# # print(events)
# for event in events:
#     print(event["messages"].content)

updates {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'x3y8ssezc', 'function': {'arguments': '{"message":"Hello there from agent","to":"919043402788"}', 'name': 'send_whatsapp_template'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 603, 'total_tokens': 632, 'completion_time': 0.03484129, 'completion_tokens_details': None, 'prompt_time': 0.049412013, 'prompt_tokens_details': None, 'queue_time': 0.063149927, 'total_time': 0.084253303}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdc77-c3cc-7b33-8566-1bcdd0486c89-0', tool_calls=[{'name': 'send_whatsapp_template', 'args': {'message': 'Hello there from agent', 'to': '919043402788'}, 'id': 'x3y8ssezc', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 603, 'output_toke

In [9]:
example_query = "send instagram message to recipientid 837220055955920 sending the message as 'Hello there from insta agent this is a sample message'"

for mode, event in agent.stream(
    {"messages": [("user", example_query)]},
    stream_mode=["updates"],
):
    print(mode, event)
# # print(events)
# for event in events:
#     print(event["messages"].content)

updates {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'p4fr11j8m', 'function': {'arguments': '{"message":"Hello there from insta agent this is a sample message","recipientid":"837220055955920"}', 'name': 'send_instagram_message'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 612, 'total_tokens': 650, 'completion_time': 0.053669228, 'completion_tokens_details': None, 'prompt_time': 0.050319064, 'prompt_tokens_details': None, 'queue_time': 0.058968206, 'total_time': 0.103988292}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdc78-32c2-7aa2-90fa-e771d6d10776-0', tool_calls=[{'name': 'send_instagram_message', 'args': {'message': 'Hello there from insta agent this is a sample message', 'recipientid': '837220055955920'}, 'id': 'p4fr11j8m', 'type': 